
# Advent of Code 2023

> The effort of using machines to mimic the human mind has always struck me as rather silly. I would rather use them to mimic something better.

-- Edsger W. Dijkstra

## Imports and definitions

In [4]:
#type: ignore
from functools import reduce
from operator import mul
from math import gcd
import re


def prod(u): 
    """Older Pythons don't have this."""
    return reduce(mul, u)


def lcm(*u):
    """Older Pythons don't have this."""
    return reduce(lambda a, b: a * b // gcd(a, b), u)


def inputfunc(day, kind='lines', testing=False):
    """Generator to read input files."""
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'commas':
            text = [x.strip() for x in open(filename).read().split(',')]
        elif kind == 'raw':
            text = open(filename).read()

        def inner():
            return func(f=text)
        return inner
    return gen

## [Day 1 - Trebuchet?!](https://adventofcode.com/2023/day/1)

In [3]:
@inputfunc(1)
def input_1(f):
    return f


def find_digits(s):
    return [int(i) for i in s if i.isdigit()]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digits(s) for s in input_1())
)

assert A == 55123


d = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

r = re.compile(f'(?=(\\d|{"|".join(d)}))')

def find_digit_names(s):
    def to_number(p):
        if p.isdigit():
            return int(p)
        else:
            return d[p]

    return [to_number(p) for p in r.findall(s)]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digit_names(s) for s in input_1())
)

assert A == 55260